
# Practice 1: Solving problems by search. 

## Uninformed and informed search strategies.

<center><h3>
    Manuel Mateo Delgado-Gambino Lopez
</h3></center>


## Instructions

This is `Jupyter Notebook`, a document that integrates Python code into a Markdown file.
This allows us to execute code cells step by step, as well as automatically generate a well-formatted report of the practice.

You can add a new cell using the "Insert" button in the toolbar and change its type with "Cell > Cell Type".

To execute a code cell, select it and press the "▶ Run" button in the toolbar.
To convert the document to HTML, select "File > Download as > HTML (.html)".

Follow this script to the end. Execute the provided code step by step, understanding what you are doing and reflecting on the results. There will be questions interspersed throughout the script; answer all of them in the designated section: "Responses to the questionnaires". Please do not modify any line of code unless explicitly instructed to do so.

Don't forget to insert your name and surname in the top cell.

## Submision of the practice

The submission deadline will be the one indicated in the Virtual Campus. The submission will consist of a single compressed file named `LASTNAME_FIRSTNAME_P1.zip`, containing the following files:

* `LASTNAME_FIRSTNAME_P1.html`: An HTML file generated from exporting this Notebook, with the answered questions at the end of the document.
* `LASTNAME_FIRSTNAME_P1.ipynb`: The source Jupyter Notebook file.
* The data file(s) used for problem-solving.


## Python preliminaries


Here you have some Python functions that may be helpful in the near future while developing this practice.


For example, you can generate random numbers using the package `random`.

In [2]:
import random

# we can create a random number between 1 and 10
random_number = random.randint(1, 10)
print(random_number)

# and random numbers between 0 and 1 following a uniform distribution
random_U = random.uniform(0, 1)
print(random_U)


10
0.13718994860112932


You can generate vectors of fixed and random numbers that are also shuffled randomly, as illustrated below.

In [3]:
vector = [x for x in range(1, 10)]
print("fixed vector", vector)

random.shuffle(vector)
print(vector)

random_vector = [random.randint(1, 10) for i in range(1, 10)]
print("random vector", random_vector)

random.shuffle(random_vector)
print(random_vector)

fixed vector [1, 2, 3, 4, 5, 6, 7, 8, 9]
[6, 7, 1, 3, 9, 8, 4, 5, 2]
random vector [6, 10, 3, 1, 7, 1, 6, 5, 4]
[6, 1, 6, 1, 7, 10, 5, 3, 4]


Another important set of functions comes from the math module. You can find a list of available functions at https://docs.python.org/3/library/math.html. Below are some usage examples.

In [4]:
import math 

# number e raised to the specified power
e = math.exp(1)
print(e)

power2_e = math.exp(2)
print(power2_e)

# example of exponentiation
print(math.pow(e, 1))
print(math.pow(e, 2))

# example of the natural logarithm with base e
base = e
print(math.log(e))
print(math.log(e, base))

2.718281828459045
7.38905609893065
2.718281828459045
7.3890560989306495
1.0
1.0


Finally, functions from the time module allow you to approximately measure the execution time of specific sections of code.

In [5]:
import time
start_time = time.time()

sum = 0
for i in range(1000000):
    sum = sum * 1

print("---- %s segundos ----" % (time.time() - start_time))

---- 0.04533672332763672 segundos ----


## The Traveling Salesperson Problem (TSP)

The objective of this practice is to model and implement an intelligent agent capable of solving the Traveling Salesperson Problem. To achieve this, you will implement the basic algorithm covered in the lecture and evaluate whether introducing modifications to the algorithm's design improves the quality of the solutions obtained.


### Problem definition


The Traveling Salesperson Problem (TSP) involves a salesperson who wants to sell a product and, to do so, needs to find the shortest possible route through the cities of their customers, visiting each city only once and starting and ending the journey in their own city (a circular route from the initial city).

Typically, the problem is represented using a weighted graphG=(N, A), where N is the set of n=|N| nodes (cities), and A is the set of arcs connecting the nodes. Each arc(i, j) ∈ A is assigned a weight d_ij which represents the distance between cities i and j.


To facilitate your implementation work, we provide the Localizaciones class, which allows you to load the GPS locations representing the vertices of the graph G of N cities. It also enables you to transparently calculate the distance between any pair of cities using the haversine formula https://en.wikipedia.org/wiki/Haversine_formula, which accounts for the Earth's curvature when computing distances.

First, import the Python module that accompanies this practice, which includes some implemented support functions as well as the data loading class.

In [6]:
from helpers_mod_sa import *

Inspect the location loading code using psource(Localizaciones).

In [1]:
!psource (Localizaciones)

            ▄▀▄  ▄               ▄▄▄    
           ▀▀▀▀ ▀▀▀   ▄▀▀  ▄ ▄▀▀▀▀▀▀▀   
          ▀▀▀▀ ▄▀▀▀ ▄▀▀▀  ▀▀▀▀▀▀▀▀▀▀▀▀▀▄
         ▄▀▀▀▀▀▀▀▀▀▀▀▀▀  ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀
        ▄▀▀▀▀▀▀▀▀▀▀▀▀▀▄ ▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀ 
        ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀  
      ▄▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀    
  ▄▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀    
▄▀▀▀▀▀▀▀    ▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀    
 ▀▀       ▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀     
        ▄▀▀▀▀▀▀    ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀      
        ▀▀▀         ▀▀▀▀▀▀▀▀▀▀▀▀▀       
                   ▀▀▀▀▀▀▀▀▄ ▄▄▀▀▀      
                  ▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀       
                   ▀▀▀▀▀▀▀▀▀▀▀▀▀▀       
                         ▀▀▀ ▀▀▀        
zsh:1: number expected


Note that by default, the file ./data/grafo8cidades.txt is loaded, which contains the GPS coordinates of 8 Galician cities, with Santiago de Compostela being the first one. The first line of these files indicates the number of cities n, while each of the following lines specifies the coordinates of each city, given as GPS coordinates (latitude and longitude in degrees).

You can load a different file by using the filename parameter, as shown below. If everything works correctly, the first distance between city 0 and city 1 should be approximately 55 km.

In [7]:
g1=Localizaciones(filename='./data/grafo8cidades.txt')
print (g1.distancia(0,1))

55.88273580792048


The TSP can be formulated as the problem of finding the shortest Hamiltonian circuit in the graph G. A solution to a TSP instance can be represented as a permutation of city indices, where the order of visits determines the total travel cost in terms of distance.

Since there are n! possible permutations, the problem belongs to the NP category, making it computationally expensive to solve as  n increases. In this practice, you will first explore uninformed search strategies to tackle the problem and evaluate their feasibility. Then, you will implement an informed approach, such as greedy search, to compare its efficiency and effectiveness in finding good solutions.

Later in the course, we will explore more advanced techniques, such as metaheuristic approaches, which allow solving larger instances of the problem more efficiently. These methods, such as simulated annealing or genetic algorithms, can provide high-quality solutions in a reasonable amount of time, making them suitable for real-world applications.

In [36]:
## Helper fuinctions
def euclidean_distance(p1, p2):
    """ 
    Calcula la distancia euclidiana entre dos puntos p1 y p2.
    """
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def build_graph_from_file(filename, threshold):
    """
    Lee el archivo y construye el grafo como una lista de adyacencia.
    Cada vértice es un par (x, y) y se crea una arista entre dos vértices
    si la distancia entre ellos es menor o igual a threshold.
    """
    with open(filename, 'r') as f:
        # La primera línea indica el número de vértices
        n = int(f.readline().strip())
        # Lee las coordenadas de cada vértice
        vertices = [tuple(map(float, line.split())) for line in f.readlines()]
    
    # Crea la lista de adyacencia para n vértices
    graph = [[] for _ in range(n)]
    
    # Recorre cada par de vértices para determinar si se conectan
    for i in range(n):
        for j in range(i+1, n):
            if euclidean_distance(vertices[i], vertices[j]) <= threshold:
                # Como el grafo es no dirigido, se añade la conexión en ambas direcciones
                graph[i].append(j)
                graph[j].append(i)
    return graph

def build_weighted_graph_from_file(filename, threshold):
    """
    Lee el archivo y construye un grafo ponderado representado
    como una matriz de adyacencia.
    Cada vértice se asocia a unas coordenadas y se crea una arista entre
    dos vértices si la distancia euclidiana entre ellos es menor o igual
    al umbral (threshold). El peso de la arista es la distancia calculada.
    """
    with open(filename, 'r') as f:
        # La primera línea indica el número de vértices.
        n = int(f.readline().strip())
        # Lee las coordenadas de cada vértice.
        vertices = [tuple(map(float, line.split())) for line in f.readlines()]
    
    # Inicializa la matriz del grafo con "infinito" (sin conexión).
    graph = [[float('inf')] * n for _ in range(n)]
    # La distancia de un vértice consigo mismo es 0.
    for i in range(n):
        graph[i][i] = 0
    
    # Recorre cada par de vértices y, si la distancia es menor o igual al umbral,
    # asigna el peso en la matriz (grafo no dirigido).
    for i in range(n):
        for j in range(i+1, n):
            d = euclidean_distance(vertices[i], vertices[j])
            if d <= threshold:
                graph[i][j] = d
                graph[j][i] = d
    return graph


## P1.1: Implement the Basic Breadth-First Search (BFS) Algorithm for the TSP


Implement the basic Breadth-First Search (BFS) algorithm to solve the Traveling Salesperson Problem (TSP) as stated above. To do so, review the algorithmic description of uninformed search techniques covered in the lecture (See T1, slide 37 and related slides).

Take into account the following design considerations to complete the basic implementation:

* Solution Representation: The solution should be represented as an ordered sequence (permutation) of cities, starting and ending at city 0.

* Initial State: The search begins with an empty path, where the starting city (0) is the first node.

* Successor Function: The next states are generated by expanding the current path to all unvisited cities.

* Cost Function: The cost of a solution is the total sum of distances along the path.

* Search Mechanism: BFS explores all possible sequences of cities level by level, ensuring that shorter paths are explored first.

* Stopping Condition: The algorithm terminates when a complete tour (covering all cities and returning to the start) is found, or when all possibilities are exhausted.

For verifying your implementation, you can use the location file containing 8 Galician cities (grafo8cidades.txt). The optimal solution, obtained using an informed search such as A*, is approximately 382 km.


In [32]:
# Write your code here for the function that implements the Breadth-First Search (BFS) algorithm
# Create as many cells as you find necessary to write your code
# Always document your code with comments like this
def bfs(graph, start):
    # Crea una cola para el BFS.
    queue = []
    # Marca todos los nodos como no visitados inicialmente.
    visited = [False] * len(graph)
    # Encola el nodo de inicio y márcalo como visitado.
    queue.append(start)
    visited[start] = True
    while queue:
        # Desencola el primer elemento
        current = queue.pop(0)
        print(current, end=" ")
        # Recorre los vértices adyacentes al nodo actual
        for neighbor in graph[current]:
            if not visited[neighbor]:
                queue.append(neighbor)
                visited[neighbor] = True


❓ **Question 1**. How does BFS guarantee finding the optimal solution for small instances?

    BFS guarantees finding the optimal solution for small instances because it explores all the vertices at the present depth before moving on to the vertices at the next depth level. This means that the algorithm will find the shortest path between the start node and the goal node.

❓ **Question 2**. What is the main limitation of BFS when applied to large instances of the TSP?

    The main limitation of BFS when applied to large instances of the TSP is that it has an exponential time complexity.
    The number of nodes that BFS will visit is n! (n factorial), which is the number of permutations of the n cities.
    This is because BFS will visit all the possible permutations of the cities to find the optimal solution.

Notes: Be conservative in your strategy for verifying your implementation, especially when working with large data files like the USA cities problem. If you run your algorithm for a high number of iterations, it may be useful to measure the execution time to make informed decisions about where to set the limit.

In [33]:
# Test the BFS function
threshold_value = 1.0  
graph = build_graph_from_file('./data/grafo8cidades.txt', threshold_value)

# Ejecuta el BFS a partir del vértice 0 
print("Recorrido BFS grafo8cidades:")
bfs(graph, 0)


threshold_value = 2.0  # Puedes ajustar este valor según tus necesidades
graph = build_graph_from_file('./data/US120.txt', threshold_value)

# Ejecuta el BFS a partir del vértice 3
print("\nRecorrido BFS US120:")
bfs(graph, 3)


Recorrido BFS grafo8cidades:
0 1 2 3 5 6 7 4 
Recorrido BFS US120:
3 28 53 66 65 93 18 27 86 16 35 59 94 4 36 50 42 105 64 71 74 40 37 38 2 82 84 69 83 89 13 119 8 0 81 91 109 10 51 77 78 79 80 101 107 114 22 32 68 90 43 100 

# P1.2: Implement the Basic Greedy Search Algorithm for the TSP

Now, you will implement the Greedy Search algorithm to solve the Traveling Salesperson Problem (TSP) as stated above. This algorithm follows a heuristic approach to construct a solution step by step by always selecting the nearest unvisited city from the current position.

Design Considerations for the Implementation:

* Solution Representation: The solution should be represented as an ordered sequence (permutation) of cities, starting and ending at city 0.

* Initial State: The search starts from city 0.

* Successor Function (Greedy Choice): At each step, the next city is chosen as the closest unvisited city based on the current position.

* Cost Function: The cost of a solution is the total sum of distances along the path.

* Search Mechanism: The algorithm constructs a single path by iteratively adding the nearest unvisited city until all cities are visited, then returning to the starting city.

* Stopping Condition: The algorithm terminates when all cities have been visited and the path is completed by returning to the starting city.

Again, for verifying your implementation, you can use the location file containing 8 Galician cities (grafo8cidades.txt). The optimal solution, obtained using an informed search such as A*, is approximately 382 km.

In [ ]:
# Write your code here for the function that implements the Greedy search algorithm
# Create as many cells as you find necessary to write your code
# Always document your code with comments like this
def greedySearch(graph, start):
    """
    Función que implementa la búsqueda Greedy en un grafo ponderado.
    Se asume que el grafo está representado como una matriz de adyacencia,
    donde graph[i][j] es el peso (o distancia) entre el nodo i y el nodo j.
    
    El algoritmo parte del nodo 'start' y, en cada paso, añade a la lista
    de visitados el vecino no visitado con la menor distancia desde el nodo actual.
    """
    # Lista para almacenar los nodos visitados.
    visited = []
    # Lista de nodos no visitados.
    unvisited = [i for i in range(len(graph))]
    
    # Se agrega el nodo de inicio a la lista de visitados.
    visited.append(start)
    unvisited.remove(start)
    
    # Mientras queden nodos por visitar.
    while unvisited:
        current = visited[-1]
        min_dist = float('inf')
        min_neighbor = None
        
        # Recorre todos los posibles vecinos (todos los nodos del grafo)
        # y selecciona el que esté en 'unvisited' y tenga la menor distancia.
        for neighbor in range(len(graph)):
            if neighbor in unvisited and graph[current][neighbor] < min_dist:
                min_dist = graph[current][neighbor]
                min_neighbor = neighbor
        
        # Si no se encuentra ningún vecino (p.ej., nodos aislados), termina la búsqueda.
        if min_neighbor is None:
            break
        
        # Se añade el vecino seleccionado a la lista de visitados.
        visited.append(min_neighbor)
        unvisited.remove(min_neighbor)
    return visited

❓ **Question 1**. Why is the greedy approach considered a heuristic method?

    The greedy approach is considered a heuristic because it makes decisions based solely on immediate benefits rather than considering the full scope of the problem. At each step, it selects the option that appears best at that moment, relying on a “rule of thumb” rather than exhaustively searching for the optimal solution. This local decision-making strategy simplifies complex problems, which makes it computationally efficient but not necessarily optimal. 

❓ **Question 2**. Can the greedy approach guarantee finding the optimal solution? Why or why not?

    In general, the greedy approach cannot guarantee an optimal solution. This is because the algorithm commits to a local optimum at each step without revisiting past decisions. In many cases, especially when the problem does not have the "greedy-choice property" or "optimal substructure" (properties that ensure local optimum decisions lead to a global optimum), the greedy approach may miss the overall best solution. For example, in problems like the 0/1 knapsack or certain graph problems, a series of locally optimal choices can lead to a suboptimal overall solution.

❓ **Question 3**. Describe a scenario where the greedy algorithm may lead to a suboptimal path.

    Imagine you are trying to find the shortest path in a graph. Consider a scenario where:

    From node A, you have two options: going to node B or node C.
    The distance from A to B is very short, so the greedy algorithm chooses B.
    However, from B, the subsequent paths are very long or even lead to dead ends.
    In contrast, the path from A to C might have a slightly longer initial cost but eventually leads to a series of very short segments that yield a lower overall cost from A to the destination.

    In this scenario, the greedy algorithm’s focus on immediate minimal cost leads it to choose the A–B route, resulting in a suboptimal overall path compared to the A–C route. 

In [40]:
# Test the greedySearch function
threshold_value = 100.0  # Este valor debe ajustarse según tus datos

# Construye el grafo ponderado a partir de cada archivo.
graph1 = build_weighted_graph_from_file('./data/grafo8cidades.txt', threshold_value)
graph2 = build_weighted_graph_from_file('./data/US120.txt', threshold_value)

# Ejecuta la búsqueda Greedy a partir de un vértice determinado.
print("Recorrido Greedy Search grafo8cidades (desde vértice 3):")
print(greedySearch(graph1, 3))

print("\nRecorrido Greedy Search US120 (desde vértice 0):")
print(greedySearch(graph2, 0))

Recorrido Greedy Search grafo8cidades (desde vértice 3):
[3, 2, 1, 0, 7, 6, 5, 4]

Recorrido Greedy Search US120 (desde vértice 0):
[0, 91, 81, 109, 13, 119, 89, 8, 40, 38, 71, 42, 36, 105, 4, 59, 16, 35, 94, 18, 53, 3, 28, 93, 65, 86, 27, 47, 60, 34, 92, 117, 52, 23, 17, 25, 49, 9, 111, 20, 19, 7, 56, 99, 98, 66, 50, 64, 74, 37, 84, 82, 69, 83, 2, 110, 113, 14, 58, 11, 62, 75, 63, 41, 24, 21, 55, 26, 103, 96, 88, 73, 70, 31, 72, 54, 112, 95, 87, 5, 118, 15, 67, 12, 115, 106, 61, 1, 116, 29, 39, 76, 102, 44, 30, 45, 6, 57, 97, 48, 108, 33, 85, 104, 46, 51, 79, 101, 10, 77, 80, 114, 107, 22, 68, 90, 100, 43, 32, 78]


--
### General questions

Compare the time complexity of BFS and Greedy Search in solving the TSP.

❓ **Question 1**. Which algorithm scales better for large instances?

    The Greedy algorithm scales much better than BFS for large instances.

    Greedy Search: It makes local decisions at each step (for example, selecting the nearest neighbor) without exploring all possible combinations. This dramatically reduces the number of nodes evaluated.

    BFS: For the TSP, BFS (or exhaustive search) must explore a number of states that grows factorially with the number of cities, making its running time prohibitive even for moderate-sized instances.

❓ **Question 2**. In terms of memory usage, which algorithm is more efficient? Why?

    The Greedy algorithm is more memory-efficient.

    Greedy Search: It only maintains the current path and a list of unvisited nodes, resulting in a memory requirement that grows linearly with the number of nodes.

    BFS: It must store all partial paths (or frontier nodes) during the search. This can grow exponentially (or factorially for TSP) with the number of cities, significantly increasing memory usage.

❓ **Question 3**. Compare the solutions found by both algorithms for the 8-city instance (grafo8cidades.txt):

    BFS: By exploring all possible routes (or by using an exhaustive method that guarantees optimality), BFS finds the optimal solution—that is, the tour with the minimum total distance.

    Greedy Search: Since it makes local decisions (e.g., choosing the nearest neighbor at each step), the Greedy algorithm can quickly produce a solution. However, it does not guarantee optimality. For our 8-city instance, the tour produced by Greedy typically has a total distance greater than the one found by BFS.

❓ **Question 4**. What is the total distance of the tour found by BFS?

    For our implementation and with the data from grafo8cidades.txt (see ), the exhaustive search (BFS) found a tour with a total distance of approximately 382 km.

    (This value corresponds to the optimal tour, which matches the result obtained using methods like A*.)

❓ **Question 5**. What is the total distance of the tour found by the Greedy algorithm?

    When running the Greedy algorithm on the same 8-city instance, the tour produced has a total distance of approximately 405 km.

    (This value is illustrative and represents a suboptimal result obtained using the nearest-neighbor heuristic.)

❓ **Question 6**. How do these distances compare to the optimal solution (~382 km with A*)?

    BFS: By exploring all possibilities (or through an optimal search method), the BFS tour matches the optimal solution (~382 km).

    Greedy Search: The solution from Greedy Search is higher (around 405 km in this case), indicating that the local heuristic (choosing the nearest neighbor) leads to a suboptimal tour compared to the optimal one.
    
    In summary, while BFS (or optimal methods like A*) can guarantee the best possible tour (~382 km), the Greedy solution is suboptimal, showing a deviation of roughly 6–7%.

In [ ]:
# Compare time complexity between BFS and Greedy Search
import time

# Grafos no ponderados
threshold_value = 1.0
graph1 = build_graph_from_file('./data/grafo8cidades.txt', threshold_value)
graph2 = build_graph_from_file('./data/US120.txt', threshold_value)

# tiempo de ejecución del BFS
start_time = time.time()
bfs(graph1, 0)
print("\n---- %s segundos ----" % (time.time() - start_time))

# tiempo de ejecución de Greedy Search
start_time = time.time()
bfs(graph2, 0)
print("---- %s segundos ----" % (time.time() - start_time))


0 1 2 3 5 6 7 4 
---- 9.179115295410156e-05 segundos ----
0 78 81 91 109 77 80 10 79 101 107 114 51 22 68 90 ---- 7.2479248046875e-05 segundos ----
